## Dealing with Missing Values in a dataframe using PySpark

In [1]:
import findspark

In [2]:
findspark.init('/home/seeram/Downloads/spark-3.2.0-bin-hadoop3.2')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Null').getOrCreate()

In [12]:
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)

In [13]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [16]:
# Remove null values from df
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [18]:
# Remove the rows there are two null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
#Remove null from any column and row
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [20]:
# Remove null values from a particular column
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
# Fill null values where the data type of column is string
df.na.fill('Fill Value').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Fill Value| null|
|emp3|Fill Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [23]:
# Fill null values where the data type of column is double 
df.na.fill(22).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| 22.0|
|emp2| null| 22.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [24]:
# To fill the null values in a particular column
df.na.fill('Name Not Known', subset=['Name']).show()

+----+--------------+-----+
|  Id|          Name|Sales|
+----+--------------+-----+
|emp1|          John| null|
|emp2|Name Not Known| null|
|emp3|Name Not Known|345.0|
|emp4|         Cindy|456.0|
+----+--------------+-----+



### Replacing null values with mean

In [44]:
from pyspark.sql.functions import mean

In [45]:
mean_value = df.select(mean(df['Sales'])).collect()

In [32]:
mean = mean_value[0][0]

In [33]:
df.na.fill(mean, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [46]:
# Replacing null values with mean value in a single line
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

